# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [1]:
import pyterrier as pt
from collections import Counter
import pandas as pd

# Initialize PyTerrier
if not pt.started():
  pt.init()

# Sample dataset
dataset = [
    {'docno': 'd1', 'text': 'do goldfish grow'}, #fragzeicehn rausparsen vorher
    {'docno': 'd2', 'text': 'quick brown fox'},
        {'docno': 'd3', 'text': 'quick white fox'},
                {'docno': 'd4', 'text': 'white quick fox'}
]

#dataset2 = [["d1", "do goldfish grow"], ["d2", "quick brown fox"], ["brown quick fox"], ["d3", "quick white fox"], ["d4", "white quick fox"]]
docs_df = pd.DataFrame([
    ["d1", "do goldfish grow"], 
    ["d2", "quick brown fox"], 
    ["d3", "brown quick fox"], 
    ["d4", "quick white fox"],
    ["d5", "white quick fox"]
    ], columns=["docno", "text"])

# Function to tokenize text into n-grams
def tokenize_ngrams(text, n=2):
    tokens = text.split()
    ngrams = [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    return ngrams
'''
# Apply n-gram tokenization to the dataset
for doc in dataset:
    ngram_tokens1 = tokenize_ngrams(doc['text'], n=1)
    ngram_tokens2 = tokenize_ngrams(doc['text'], n=2)
    ngram_tokens = ngram_tokens1 + ngram_tokens2
    token_counts = Counter(ngram_tokens)
    doc['toks'] = token_counts
    del doc['text']  # Remove the 'text' field as it's not needed anymore
'''
# Initialize the IterDictIndexer with pretokenised set to True
indexer = pt.DFIndexer("./pretokindex", overwrite=True) #blocks = true
index_ref = indexer.index(docs_df["text"], docs_df["docno"])
print(f"Indexing complete: {index_ref}")
index_ref.toString()
index = pt.IndexFactory.of(index_ref)

print(index.getCollectionStatistics())
'''
for term, le in index.getLexicon():
    print(term) 
    print(le.getFrequency())
    '''


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Indexing complete: <org.terrier.querying.IndexRef at 0x7ab28e9147c0 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x611d04e01f80 at 0x7ab28e89fb50>>
Number of documents: 5
Number of terms: 6
Number of postings: 14
Number of fields: 0
Number of tokens: 14
Field names: []
Positions:   false



'\nfor term, le in index.getLexicon():\n    print(term) \n    print(le.getFrequency())\n    '

In [1]:

# Function to convert n-grams into matchop query language
def ngram_to_matchop(query, n=2):
    ngrams = tokenize_ngrams(query, n)
    matchop_query = ' '.join([f'#od1({ngram})' for ngram in ngrams])
    return matchop_query

# Define the MatchopQueryTransform transformer
class MatchopQueryTransform(pt.Transformer):
    def __init__(self, n=2):
        self.n = n

    def transform(self, topics):
        topics['query'] = topics['query'].apply(lambda x: ngram_to_matchop(x, self.n))
        return topics

# Initialize the BatchRetrieve with the indexed reference and BM25
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25")

# Create a query pipeline that includes the n-gram tokenization in matchop format
matchop_tokenize = MatchopQueryTransform(n=1)
query_toks = pt.apply.query(lambda row: ngram_to_matchop(row['query'], n=1))
retr_pipe = query_toks >> bm25

# Sample queries
queries = pd.DataFrame([
    {'qid': 'q1', 'query': 'do goldfish grow'},
    {'qid': 'q2', 'query': 'quick brown fox'}
])

# Retrieve results
run = retr_pipe.transform(queries)
print(run.head(10))

NameError: name 'pt' is not defined

In [47]:
query = 'grow'
bm25 = pt.BatchRetrieve(index)
run = bm25.search(query)
print(run.head(10))
# Retrieve documents
#results = index.search(query)

# Print retrieved documents
#for doc in results:
 #   print("Document:", doc)

20:13:53.148 [main] ERROR org.terrier.structures.Index - This index (/workspaces/ir-lab-sose-2024-needthegrade/baseline-retrieval-system/pretokindex/data.properties) doesnt have an index structure called lexicon: property index.lexicon.class not found
20:13:53.150 [main] ERROR org.terrier.structures.Index - Valid structures are: [meta-inputstream, lexicon-valuefactory, document, meta, direct, direct-inputstream, document-inputstream, document-factory, lexicon-keyfactory]
20:13:53.152 [main] ERROR org.terrier.structures.Index - This index (/workspaces/ir-lab-sose-2024-needthegrade/baseline-retrieval-system/pretokindex/data.properties) doesnt have an index structure called inverted: property index.inverted.class not found
20:13:53.153 [main] ERROR org.terrier.structures.Index - Valid structures are: [meta-inputstream, lexicon-valuefactory, document, meta, direct, direct-inputstream, document-inputstream, document-factory, lexicon-keyfactory]
20:13:53.155 [main] ERROR org.terrier.structur

JavaException: JVM exception occurred: java.lang.NullPointerException

### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

Create a custom tokenizer method.
This method takes a String as an argument and returns a List of all tokens, including ngrams between 1 and 3 words.

Create the Iterator for our dataset. This is an Iterator where every element is a dict which contains a text and a docno.
See the example print below.

### Step 3: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [3]:
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", verbose = True)

### Step 4: Create the Run


In [4]:
print('Now we do the retrieval...')
#run = bm25(test_dataset.get_topics('text'))
#run = bm25([{"qid": "0", "query" : "retrieval systems"}])
#print('Done. Here are the first 10 entries of the run')
#run.head(10)


# tokenize ngram the queries
# Create a query pipeline that includes the n-gram tokenization
#ngram_tokenize = NgramTokenizeTransform(n=2)
retr_pipe = ngram_tokenize >> bm25

import pandas as pd
# Sample queries
queries = pd.DataFrame([
    {'qid': 'q1', 'query': 'do goldfish grow?'},
    {'qid': 'q2', 'query': 'quick brown fox'}
])

# Retrieve results
run = retr_pipe(queries)
run.head(10)
results = retr_pipe.transform(queries)
print(results)

Now we do the retrieval...


TypeError: ValueError() takes no keyword arguments

In [9]:
#index = pt.IndexFactory.of(index_ref)
meta = index.getMetaIndex()

# List of metadata keys
meta_keys = ['docno', 'text']  # Adjust this list based on your meta fields

# Function to print document attributes
def print_doc_attributes(docid):
    attributes = {key: meta.getItem(key, docid) for key in meta_keys}
    print(f"Attributes for docid {docid}: {attributes}")

# Example: Print attributes for the first 5 documents
for docid in range(3):
    print_doc_attributes(docid)

# Function to print document attributes by docno
def print_doc_attributes_by_docno(docno):
    try:
        docid = meta.getDocument("docno", docno)
        print_doc_attributes(docid)
    except KeyError:
        print(f"Document with docno {docno} not found.")

# List of specific docnos to retrieve
#docnos = ['W05-0704']

# Retrieve and print attributes for each specified docno
#for docno in docnos:
 #   print_doc_attributes_by_docno(docno)

Attributes for docid 0: {'docno': '0', 'text': ''}
Attributes for docid 1: {'docno': '1', 'text': ''}
Attributes for docid 2: {'docno': '2', 'text': ''}


### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [ ]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
